# Unit Commitment

TODO

In [6]:
;tree -a

.
├── .ipynb_checkpoints
│   ├── facility-checkpoint.ipynb
│   ├── gurobi-checkpoint.env
│   ├── knapsack-checkpoint.ipynb
│   ├── preliminaries-checkpoint.ipynb
│   ├── stab-checkpoint.ipynb
│   ├── tsp-checkpoint.ipynb
│   └── uc-checkpoint.ipynb
├── facility.ipynb
├── gurobi.env
├── knapsack.ipynb
├── preliminaries.ipynb
├── stab.ipynb
├── tsp.ipynb
└── uc.ipynb

1 directory, 14 files
